In [1]:
# # Loading data and fitting a choice model

# The package expects your data to live in a single .mat file which should contain a struct called `rawdata`. Each element of `rawdata` should have data for one behavioral trial and `rawdata` should contain the following fields with the specified structure:

# - `rawdata.leftbups`: row-vector containing the relative timing, in seconds, of left clicks on an individual trial. 0 seconds is the start of the click stimulus.
# - `rawdata.rightbups`: row-vector containing the relative timing in seconds (origin at 0 sec) of right clicks on an individual trial. 0 seconds is the start of the click stimulus.
# - `rawdata.T`: the duration of the trial, in seconds. The beginning of a trial is defined as the start of the click stimulus. The end of a trial is defined based on the behavioral event “cpoke_end”. This was the Hanks convention.
# - `rawdata.pokedR`: `Bool` representing the animal choice (1 = right).
# - `rawdata.sessbnd`: `Bool` indicating the first trial of every session

# ## Fitting the model

# Once your data is correctly formatted and you have the package added in julia, you are ready to fit the model. You need to write a slurm script to use spock's resources and a .jl file to load the data and fit the model. See examples of each below. These files are also located in the package in the `examples` directory.

# ### Example slurm script

# This will start a job called `fit_choice_model`. Output will be written to a log file called `fit_choice_model.out`. This will run on the `Brody` partition of spock for 12 hours, using 44 cores and 64 GB of memory. You'll notice that we load the julia module (like we did when we added the package) and then we call julia (`-p 44` uses the 44 cores) and ask it to run the .jl file.

# ```
# #!/usr/bin/env bash
#
# #SBATCH -J 'fit_choice_model'
# #SBATCH -o ../logs/fit_choice_model.out
# #SBATCH -p Brody
# #SBATCH --time=12:00:00
# #SBATCH --mem=64000
# #SBATCH -c 44
#
# module load julia/1.0.0
# julia -p 44 ./fit_choice_model.jl
# ```

In [2]:
# ### Example .jl file
# Blah blah blah

using pulse_input_DDM, Flatten

In [5]:
using MAT
data = read(matopen("../choice model/example_matfile.mat"), "rawdata")


Dict{String,Any} with 5 entries:
  "rightbups" => Any[[0.0; 0.000392216; … ; 0.350424; 0.374483] [0.0; 0.0187241…
  "T"         => Any[0.39 0.48 … 0.41 0.99]
  "leftbups"  => Any[[0.0; 0.0193924; … ; 0.383063; 0.383503] [0.0; 0.0276688; …
  "pokedR"    => Any[false false … false true]
  "sessbnd"   => Any[true false … false false]

In [8]:
haskey(data, "sessbnd")
T = vec(data["T"])
    L = vec(map(x-> vec(collect(x)), data[collect(keys(data))[occursin.("left", collect(keys(data)))][1]]))
    R = vec(map(x-> vec(collect(x)), data[collect(keys(data))[occursin.("right", collect(keys(data)))][1]]))
    choices = vec(convert(BitArray, data["pokedR"]))

10-element BitArray{1}:
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [9]:
length(choices)

10

In [3]:
# ### Load some data
# Blah blah blah

data = load_choice_data("../choice model/example_matfile.mat");

In [4]:
# ### Set options for optimization
# Blah blah blah

n = 53

options, x0 = create_options_and_x0(modeltype="bing")

(choiceoptions
  fit: Array{Bool}((15,)) Bool[true, true, true, true, true, true, true, true, false, false, false, false, false, false, true]
  lb: Array{Float64}((15,)) [0.0, 8.0, -5.0, 0.0, 0.0, 0.01, 0.005, 0.0, 0.0, 0.0, -2.5, -2.5, 0.0, 0.0, -30.0]
  ub: Array{Float64}((15,)) [2.0, 30.0, 5.0, 100.0, 2.5, 1.2, 1.0, 1.0, 10.0, 20.0, 2.5, 2.5, 1.0, 1.0, 30.0]
, [1.97759, 18.5116, 0.13746, 33.8513, 1.11076, 0.271462, 0.914436, 0.982399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.9219])

In [6]:
# ### Load some data
# Blah blah blah
save_file = "../choice model/example_results.mat"

#if you've already ran the optimization once and want to restart from where you stoped, this will reload those parameters
if isfile(save_file)
    θ, options = reload_choice_model(save_file)
else
    θ = Flatten.reconstruct(θchoice(), x0)
end

θchoice{θz{Float64},Float64}
  θz: θz{Float64}
  bias: Float64 0.0
  lapse: Float64 0.01


In [7]:
# ### Optimize stuff
# Blah blah blah

model, = optimize(θ, data, options; iterations=5, outer_iterations=1)

Fminbox
-------
Initial mu = 0.000239451

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000239451

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.615269e+00     9.828961e+01
 * time: 5.412101745605469e-5

Exiting inner optimizer with x = [0.12345655322552201, 14.998955306903198, -0.13726257383761056, 19.99805424621316, 0.6951082089315415, 1.1588961674105702, 0.0063088058501644005, 0.6814442212446054, 0.5003949673659975]
Current distance to box: 0.00130881
Decreasing barrier term μ.



(choiceDDM{θchoice{θz{Float64},Float64},Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}},1}}
  θ: θchoice{θz{Float64},Float64}
  data: Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [1.23e-01, 1.50e+01, -1.37e-01,  ...]
    Minimum:   4.955724e+00

 * Found with
    Algorithm:     Fminbox with BFGS
    Initial Point: [1.00e-01, 1.50e+01, -1.00e-01,  ...]

 * Convergence measures
    |x - x'|               = 9.35e-01 ≰ 1.0e-10
    |x - x'|/|x'|          = 3.73e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.66e+00 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.37e-01 ≰ 1.0e-09
    |g(x)|                 = 2.03e+00 ≰ 1.0e-03

 * Work counters
    Seconds run:   9  (vs limit 170000)
    Iterations:    1
    f(x) calls:    27

In [8]:
# ### Compute Hessian and the confidence interavls
# Blah blah blah

H = Hessian(model)
CI, = CIs(H);

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 8.46979164769424
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


In [10]:
# ### Save results
# Blah blah blah

save_choice_model(save_file, model, options, CI)

In [10]:
# ### Getting help
# To get more details about how any function in this package works, in julia you can type `?` and then the name of the function. Documentation will display in the REPL.